In [1]:
import urllib2
from unidecode import unidecode
from bs4 import BeautifulSoup
import re

In [2]:
#the starting page for scraping (and or crawling?)
darklyrics_kamelot_url = "http://www.darklyrics.com/k/kamelot.html"

In [3]:
# open the page with urllib2
darklyrics_kamelot_page = urllib2.urlopen(darklyrics_kamelot_url)
# scrap using beautifulsoup
darklyrics_kamelot_soup = BeautifulSoup(darklyrics_kamelot_page, "lxml")

In [4]:
# see the whole page (with prettify)
print darklyrics_kamelot_soup.prettify()

# wanna see the all the links in the page?
# for a in darklyrics_kamelot_soup.find_all('a'):
#     print a.get("href")

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html lang="en" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml">
 <head>
  <title>
   KAMELOT lyrics
  </title>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content='KAMELOT lyrics - 160 song lyrics from 12 albums, including "Haven" (2015).' name="description"/>
  <meta content="KAMELOT, lyrics, discography, albums, songs" name="keywords"/>
  <link href="../dark.css" rel="stylesheet" type="text/css"/>
  <script type="text/javascript">
   var _gaq = _gaq || [];
  _gaq.push(['_setAccount', 'UA-20396473-1']);
  _gaq.push(['_trackPageview']);

  (function() {
    var ga = document.createElement('script'); ga.type = 'text/javascript'; ga.async = true;
    ga.src = ('https:' == document.location.protocol ? 'https://ssl' : 'http://www') + '.google-analytics.com/ga.js';
    var s = document.getElementsByTagName('script')[0]; s.parentNode

In [5]:
# only need to take the link with /lyrics/kamelot/....html in it. use regex.
# put the urls in a list
all_lyric_links = darklyrics_kamelot_soup.find_all('a', attrs={'href': re.compile("\/lyrics\/kamelot\/.*\.html")})
lyrics_url = []
for link in all_lyric_links:
    lyrics_url.append(link.get("href"))

#see the urls    
lyrics_url

['../lyrics/kamelot/eternity.html#1',
 '../lyrics/kamelot/eternity.html#2',
 '../lyrics/kamelot/eternity.html#3',
 '../lyrics/kamelot/eternity.html#4',
 '../lyrics/kamelot/eternity.html#5',
 '../lyrics/kamelot/eternity.html#6',
 '../lyrics/kamelot/eternity.html#7',
 '../lyrics/kamelot/eternity.html#8',
 '../lyrics/kamelot/eternity.html#9',
 '../lyrics/kamelot/eternity.html#10',
 '../lyrics/kamelot/eternity.html#11',
 '../lyrics/kamelot/dominion.html#1',
 '../lyrics/kamelot/dominion.html#2',
 '../lyrics/kamelot/dominion.html#3',
 '../lyrics/kamelot/dominion.html#4',
 '../lyrics/kamelot/dominion.html#5',
 '../lyrics/kamelot/dominion.html#6',
 '../lyrics/kamelot/dominion.html#7',
 '../lyrics/kamelot/dominion.html#8',
 '../lyrics/kamelot/dominion.html#9',
 '../lyrics/kamelot/dominion.html#10',
 '../lyrics/kamelot/dominion.html#11',
 '../lyrics/kamelot/sigeperilous.html#1',
 '../lyrics/kamelot/sigeperilous.html#2',
 '../lyrics/kamelot/sigeperilous.html#3',
 '../lyrics/kamelot/sigeperilous.h

In [6]:
# dammit it looks like the href urls are not complete (there're ../). well I can change it with regex 
# since it's dumb to traverse the *same* page 10++ times, let's prune the #1, #2, ...
# so not cool, too much manual tinkering :/
for i in range(len(lyrics_url)):
    lyrics_url[i] = re.sub(r'\.\.','http://www.darklyrics.com',lyrics_url[i])
    lyrics_url[i] = re.sub(r'\#.*','',lyrics_url[i])

# remove duplicate in the list
lyrics_url = list(set(lyrics_url))
lyrics_url

['http://www.darklyrics.com/lyrics/kamelot/ghostopera.html',
 'http://www.darklyrics.com/lyrics/kamelot/silverthorn.html',
 'http://www.darklyrics.com/lyrics/kamelot/sigeperilous.html',
 'http://www.darklyrics.com/lyrics/kamelot/dominion.html',
 'http://www.darklyrics.com/lyrics/kamelot/epica.html',
 'http://www.darklyrics.com/lyrics/kamelot/thefourthlegacy.html',
 'http://www.darklyrics.com/lyrics/kamelot/onecoldwintersnight.html',
 'http://www.darklyrics.com/lyrics/kamelot/theblackhalo.html',
 'http://www.darklyrics.com/lyrics/kamelot/haven.html',
 'http://www.darklyrics.com/lyrics/kamelot/karma.html',
 'http://www.darklyrics.com/lyrics/kamelot/poetryforthepoisoned.html',
 'http://www.darklyrics.com/lyrics/kamelot/eternity.html']

In [7]:
# make a dictionary to store the lyrics
# why dictionary? because later I need to identify albums: pre-Roy, Roy, and Tommy
full_albums = {}
for url in lyrics_url:
    # extract the album name from url, open page, and extract the lyrics on the album
    album_name = re.search(r'kamelot\/(.*)\.html', url).group(1)
    url_page = urllib2.urlopen(url)    
    url_soup = BeautifulSoup(url_page, "lxml")
    the_lyrics = url_soup.find_all("div", {"class" : "lyrics"})
    
    # remove the junk information, split the whole text by the songs, and store in dict
    for x in the_lyrics:
        x = str(x)
        x = re.sub(r"\<.*?\>","",x)
        album = re.split(r'[0-9].*\n', x)
        full_albums.update({album_name : album})

In [11]:
#example of the lyrics, junk information still exist. need to be proceed later
print full_albums['theblackhalo'][4]

If my soul could revive
From my carnal remains
What does it matter to me
If it all fades to black
If I'm born once again
Then no one really is free
How could I be condemned
For the things that I've done
If my intentions were good
I guess I'll never know
Some things under the sun
Can never be understood

How can we believe in heaven
Human reason counters all
Ideas of a soul society
My life is just a fragment
Of the universe and all
There must be more than I can see

In the dark we're the same
In the concept of time
We're like a grain in the sand
And we strive for the flame
As if death was our aim
Cause we cannot understand

How I wish there was heaven
All for one and one for all
A flawless soul society
Our lives are just a fragment
Of the universe and all
There may be more than we can see


